In [11]:
import pandas as pd
import numpy as np
import os

In [ ]:
model = 'geneva_200m'
params_file = 'swirl_03'
output_folder = r'../Outputs'

lvl0_filename = f'{model}_{params_file}_lvl0.csv'
output_filename = f'{model}_{params_file}_lvl1.csv'

In [4]:
level0_csv_path = os.path.join(output_folder, lvl0_filename)
level0_data = pd.read_csv(level0_csv_path)

In [51]:
level1_csv_path = os.path.join(output_folder, output_filename)

In [ ]:
def identify_vortex(data, id_eddy, threshold):
    #create filter
    mask = (
        (data['date'] == data['date'][id_eddy]) &
        (data['rotation_direction'] == data['rotation_direction'][id_eddy]) &
        (np.sqrt((data['xc'] - data['xc'][id_eddy])**2 + (data['yc'] - data['yc'][id_eddy])**2) < threshold)
    )
    
    return  level0_data.loc[mask] #apply filter

In [48]:
dist_threshold = 2 # in number of cells

df_catalogue_level1 = pd.DataFrame(columns=[
    'id',
    'id_lvl0',
    'date',
    'depth_min_[m]',
    'depth_max_[m]',
    'xc_mean',
    'yc_mean',
    'surface_area_mean_[m2]',
    'volume_[m3]',
    'rotation_direction',
    'kinetic_energy_[MJ]'
])
    
id_level1=0
idx_already_aggregated = []
for idx in level0_data['id']:
    if idx in idx_already_aggregated:
        continue

    aggregated_data = identify_vortex(level0_data, idx)
    
    df_catalogue_level1.loc[id_level1] = [
        id_level1,
        aggregated_data['id'].tolist(),
        aggregated_data['date'][idx],
        aggregated_data['depth_[m]'].max(),
        aggregated_data['depth_[m]'].min(),
        aggregated_data['xc'].mean(),
        aggregated_data['yc'].mean(),
        aggregated_data['surface_area_[m2]'].mean(),
        aggregated_data['volume_slice_[m3]'].sum(),
        aggregated_data['rotation_direction'][idx],
        aggregated_data['kinetic_energy_[MJ]'].sum()
    ]
    id_level1 += 1
    idx_already_aggregated.extend(aggregated_data['id'].tolist())

In [52]:
df_catalogue_level1.to_csv(level1_csv_path, index=False)

TO DO 
- Check depth consistency